# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean. |


In [46]:
# Write your answer to Task 1 here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('production_data.csv')

print(data.info())
print(data.isna().sum())

data['mixing_time'] = data['mixing_time'].fillna(data['mixing_time'].mean())

print(data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   batch_id               2000 non-null   int64  
 1   production_date        2000 non-null   object 
 2   raw_material_supplier  2000 non-null   int64  
 3   pigment_type           2000 non-null   object 
 4   pigment_quantity       2000 non-null   float64
 5   mixing_time            1982 non-null   float64
 6   mixing_speed           2000 non-null   object 
 7   product_quality_score  2000 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 125.1+ KB
None
batch_id                  0
production_date           0
raw_material_supplier     0
pigment_type              0
pigment_quantity          0
mixing_time              18
mixing_speed              0
product_quality_score     0
dtype: int64
batch_id                 0
production_date          0
raw_material_s

In [47]:
data['production_date'] = pd.to_datetime(data['production_date'], errors='coerce')

data['raw_material_supplier'] = data['raw_material_supplier'].replace({'national_supplier': 1, 'international_supplier': 2})
print(data['raw_material_supplier'].value_counts())

data['pigment_type'] = data['pigment_type'].str.lower()
print(data['pigment_type'].value_counts())

print(data['pigment_quantity'].agg(['min', 'max']))

print(data['mixing_speed'].value_counts())
data.loc[~data['mixing_speed'].isin(['Low', 'Medium', 'High']), 'mixing_speed'] = 'Not Specified'
print(data['mixing_speed'].value_counts())

print(data['product_quality_score'].agg(['min', 'max']))


2    1300
1     700
Name: raw_material_supplier, dtype: int64
type_c    737
type_b    658
type_a    605
Name: pigment_type, dtype: int64
min    19.413042
max    58.727253
Name: pigment_quantity, dtype: float64
Medium    678
High      650
Low       638
-          34
Name: mixing_speed, dtype: int64
Medium           678
High             650
Low              638
Not Specified     34
Name: mixing_speed, dtype: int64
min     2.227802
max    10.000000
Name: product_quality_score, dtype: float64


In [48]:
clean_data = data

print(clean_data.info())
print(clean_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   batch_id               2000 non-null   int64         
 1   production_date        2000 non-null   datetime64[ns]
 2   raw_material_supplier  2000 non-null   int64         
 3   pigment_type           2000 non-null   object        
 4   pigment_quantity       2000 non-null   float64       
 5   mixing_time            2000 non-null   float64       
 6   mixing_speed           2000 non-null   object        
 7   product_quality_score  2000 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 125.1+ KB
None
   batch_id production_date  ...  mixing_speed product_quality_score
0         1      2024-06-25  ...          High              7.165102
1         2      2023-11-23  ...          High              6.849126
2        

# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [49]:
aggregated_data = clean_data.groupby('raw_material_supplier').agg(
    avg_product_quality_score=('product_quality_score', 'mean'),
    avg_pigment_quantity=('pigment_quantity', 'mean')
).reset_index().round(2)

print(aggregated_data)

   raw_material_supplier  avg_product_quality_score  avg_pigment_quantity
0                      1                       8.02                 44.73
1                      2                       5.97                 34.91


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [50]:
# Write your answer to Task 3 here

filtered_data = data[(data['raw_material_supplier'] == 2) & (data['pigment_quantity'] > 35)]

avg_product_quality_score = filtered_data['product_quality_score'].mean()
avg_pigment_quantity = filtered_data['pigment_quantity'].mean()

pigment_data = pd.DataFrame({
    'raw_material_supplier': [2],
    'pigment_quantity': [avg_pigment_quantity.round(2)],
    'avg_product_quality_score': [avg_product_quality_score.round(2)]
})

print(pigment_data)

   raw_material_supplier  pigment_quantity  avg_product_quality_score
0                      2             39.01                       5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [51]:
# Write your answer to Task 4 here

product_quality_score_mean = data['product_quality_score'].mean()
product_quality_score_sd = data['product_quality_score'].std()
pigment_quantity_mean = data['pigment_quantity'].mean()
pigment_quantity_sd = data['pigment_quantity'].std()

corr_coef = data[['pigment_quantity', 'product_quality_score']].corr().iloc[0, 1]

product_quality = pd.DataFrame({
    'product_quality_score_mean': [round(product_quality_score_mean, 2)],
    'product_quality_score_sd': [round(product_quality_score_sd, 2)],
    'pigment_quantity_mean': [round(pigment_quantity_mean, 2)],
    'pigment_quantity_sd': [round(pigment_quantity_sd, 2)],
    'corr_coef': [round(corr_coef, 2)]
})

print(product_quality)

   product_quality_score_mean  ...  corr_coef
0                        6.68  ...       0.49

[1 rows x 5 columns]
